<a href="https://colab.research.google.com/github/DataTak/finacial_data_analysis/blob/main/%EC%A6%9D%EA%B6%8C%EC%82%AC%EB%A6%AC%ED%8F%AC%ED%8A%B8%ED%81%AC%EB%A1%A4%EB%A7%81_paxnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 증권사 리포트 크롤링
- Paxnet의 증권사리포트의 종목리포트를 수집하여 데이터 프레임 생성.
- 데이터 프레임을 `report.csv`로 저장하기

- 증권사 리포트의 종목리포트 탭 <br>
`https://www.paxnet.co.kr/stock/report/report?wlog_rpt=jm&menuCode=2222`
    - 페이지네이션이 되어 있고, 페이지 변경시, currentPageNo가 변경됨.<br>
    https://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo=2&reportId=0&searchKey=stock&searchValue=
- 데이터프레임의 구조 <br>
종목명 / 제목 / 적정가격 / 투자의견 / 제공출처 / 작성일    


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from datetime import datetime, timedelta


## 증권사 리포트 목록 불러오기

In [3]:
url = 'https://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo=1&reportId=0&searchKey=stock&searchValue='
response = requests.get(url)

In [26]:
soup = BeautifulSoup(response.text, 'lxml')
list_stocks = soup.find_all('li')

- 데이터 프레임 구조
    - 데이터 위치: `<li class='board-lish-th'>`

## 내용 파싱 및 데이터프레임으로 저장 함수

In [32]:
def stock_info(soup):
    data = []
    # 모든 <li> 태그를 찾기
    list_items = soup.find_all("li")

    for item in list_items:
        try:
            # 종목명
            stock_name = item.find_all("div")[0].text.strip()

            # 제목
            title = item.find_all("div")[1].text.strip()

            # 적정가격
            price = item.find_all("div")[2].text.strip().replace("적정가격", "").strip()

            # 투자의견
            investment_opinion = item.find_all("div")[3].text.strip()

            # 제공출처
            source = item.find_all("div")[4].text.strip()

            # 작성일
            date = item.find_all("div")[5].text.strip()

            # 데이터 저장
            data.append([stock_name, title, price, investment_opinion, source, date])

        except IndexError:
            # 데이터가 없는 경우 스킵
            continue
    df = pd.DataFrame(data, columns=["종목", "제목", "적정가격", "투자의견", "제공출처", "작성일"])[1:]
    df['종목'] = df['종목'].str.replace('[', '').str.replace(']', '')
    return df

## 여러 페이지의 정보 가지고 오기

In [40]:
stock_data = pd.DataFrame(columns=["종목", "제목", "적정가격", "투자의견", "제공출처", "작성일"])

In [41]:
for page in range(1, 10):  # 종료 페이지 설정하기
    url = f'https://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={page}&reportId=0&searchKey=stock&searchValue='
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    stock_data = pd.concat([stock_data, stock_info(soup)], ignore_index=True)
    time.sleep(1)

In [42]:
stock_data.tail(1)

,종목,제목,적정가격,투자의견,제공출처,작성일
269,LG이노텍,"3Q 영업이익 부진, 밸류에이션에 초점",300000원,매수,대신증권,2024.09.24


In [43]:
stock_data.head(1)

,종목,제목,적정가격,투자의견,제공출처,작성일
0,인탑스,로봇 EMS 기대감,"30,000원",매수,LS증권,2024.10.28


In [44]:
stock_data.to_csv('report.csv', index=False)